# Student Performance Regression Analysis
- This notebook is to perform eda analysis on the student performance data and identify a relationship with the 
- individual test scores as related to the student demographics
- Data Collection is from Kaggle repository https://www.kaggle.com/datasets/spscientist/student-performance-in-exams

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings


In [6]:
#df = pd.read_csv('data/StudentsPerformance.csv')
df = pd.read_csv('/Users/ramkisundararaman/mlproject/data/StudentsPerformance.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


- Statistical Description of Data

In [7]:
df.describe()

,math score,reading score,writing score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


#### Observations
- The data frame has 1000 rows divided into 8 columns. There are three numerical columns indicating the score of the student in Math, Reading and Writing. There are 5 categorical variables indicating the student demographic. There are no missing values.

--  Display the categorical columns and their unique values

In [8]:
categorical_columns = [col for col in df.columns if df[col].dtype =='O']
numerical_columns = [col for col in df.columns if df[col].dtype != 'O']
print(len(categorical_columns))
print(categorical_columns)
print (len(numerical_columns))
print(numerical_columns)

5
['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
3
['math score', 'reading score', 'writing score']


In [9]:
def Print_unique_values():
    for col in categorical_columns:
       print('Unique Values in Category ' + col + ' is:'+ str(df[col].nunique()))
    for col in numerical_columns:
        print('Unique Values in Category ' + col + 'is:'+  str(df[col].nunique()))


Print_unique_values()                        

Unique Values in Category gender is:2
Unique Values in Category race/ethnicity is:5
Unique Values in Category parental level of education is:6
Unique Values in Category lunch is:2
Unique Values in Category test preparation course is:2
Unique Values in Category math scoreis:81
Unique Values in Category reading scoreis:72
Unique Values in Category writing scoreis:77


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


- Split the Date Frame into X and Y. The Model will predict a student's Math_score based on the demographics and scores in writing and reading

In [11]:
X = df.drop(columns=['math score'],axis=1)
Y = df['math score']



In [12]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features)
    ]
)
X = preprocessor.fit_transform(X)

- Split the data set into Training and Testing data

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [14]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square= r2_score(true,predicted)
    return mae,rmse,r2_square

In [15]:
models = {
    "Linear Regression" : LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGB Regressor":XGBRegressor(),
    "Cat Boost Regressor":CatBoostRegressor(),
    "Ada Boost Regressor" : AdaBoostRegressor()
}
model_list=[]
r2_list=[]  ## List to capture PErformance Metrics

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,Y_train)
    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)
    model_train_mae, model_train_rmse,model_train_r2 = evaluate_model(Y_train,Y_train_pred)
    model_test_mae, model_test_rmse,model_test_r2 = evaluate_model(Y_test,Y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model performance for Training Set")
    print("-- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("-- Mean Absoluted Error:{:.4f}".format(model_train_mae))
    print("-- Mean Absoluted Error:{:.4f}".format(model_train_mae))
    print("-- R2 Score:{:.4f}".format(model_train_r2))
    print("--------------------------------------------------------------------")
    print("Model performance score for Test data")
    print("-- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("-- Mean Absoluted Error:{:.4f}".format(model_test_mae))
    print("-- Mean Absoluted Error:{:.4f}".format(model_test_mae))
    print("-- R2 Score:{:.4f}".format(model_test_r2))

    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')

Linear Regression
Model performance for Training Set
-- Root Mean Squared Error: 5.3263
-- Mean Absoluted Error:4.2689
-- Mean Absoluted Error:4.2689
-- R2 Score:0.8742
--------------------------------------------------------------------
Model performance score for Test data
-- Root Mean Squared Error: 5.4252
-- Mean Absoluted Error:4.2263
-- Mean Absoluted Error:4.2263
-- R2 Score:0.8790


Lasso
Model performance for Training Set
-- Root Mean Squared Error: 6.5938
-- Mean Absoluted Error:5.2063
-- Mean Absoluted Error:5.2063
-- R2 Score:0.8071
--------------------------------------------------------------------
Model performance score for Test data
-- Root Mean Squared Error: 6.5197
-- Mean Absoluted Error:5.1579
-- Mean Absoluted Error:5.1579
-- R2 Score:0.8253


Ridge
Model performance for Training Set
-- Root Mean Squared Error: 5.3233
-- Mean Absoluted Error:4.2650
-- Mean Absoluted Error:4.2650
-- R2 Score:0.8743
-------------------------------------------------------------------

Random Forest Regressor
Model performance for Training Set
-- Root Mean Squared Error: 2.3067
-- Mean Absoluted Error:1.8401
-- Mean Absoluted Error:1.8401
-- R2 Score:0.9764
--------------------------------------------------------------------
Model performance score for Test data
-- Root Mean Squared Error: 6.0381
-- Mean Absoluted Error:4.6941
-- Mean Absoluted Error:4.6941
-- R2 Score:0.8502


XGB Regressor
Model performance for Training Set
-- Root Mean Squared Error: 1.0073
-- Mean Absoluted Error:0.6875
-- Mean Absoluted Error:0.6875
-- R2 Score:0.9955
--------------------------------------------------------------------
Model performance score for Test data
-- Root Mean Squared Error: 6.4733
-- Mean Absoluted Error:5.0577
-- Mean Absoluted Error:5.0577
-- R2 Score:0.8278


Learning rate set to 0.039525
0:	learn: 14.5987177	total: 57.4ms	remaining: 57.3s
1:	learn: 14.2251886	total: 57.8ms	remaining: 28.9s
2:	learn: 13.8866124	total: 58.3ms	remaining: 19.4s
3:	learn: 13.5235688	tot

In [16]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
                                                                                                 

,Model Name,R2_Score
2,Ridge,0.880593
0,Linear Regression,0.879047
6,Cat Boost Regressor,0.851632
4,Random Forest Regressor,0.850173
7,Ada Boost Regressor,0.846390
5,XGB Regressor,0.827797
1,Lasso,0.825320
3,Decision Tree,0.712088
